# Tryout basic chains

In [ ]:
from IPython.display import display, Markdown, JSON
import dotenv
dotenv.load_dotenv()

## Only openai

In [ ]:
import openai
messages=[
     {"role": "system", "content": "You are a code expert"}
]

conversation_total_tokens = 0
while True:
    message = input ( "Human: ")
    if message=='exit':
        print (f" {conversation_total_tokens} tokens used in total in this conversation" )
        break
    if message:
        messages.append(
            {"role": "user" , "content": message}
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    reply = response.choices[0].message.content
    total_tokens = response.usage['total_tokens']
    conversation_total_tokens += total_tokens
    print ("ChatGPT:")
    display(Markdown(reply))
    print (f"\nIn {total_tokens} tokens used" )
    messages. append( {"role": "assistant", "content": reply})

## Langchain

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, OpenAIChat

In [ ]:
prefix_messages=[
     {"role": "system", "content": "You are a code expert"}
]

llm = OpenAIChat(
    model_name= 'gpt-3.5-turbo', 
    temperature=0.0,
    prefix_messages=prefix_messages,
    max_tokens=256
)

In [ ]:
template ="""
You are an expert of cooking and name entity recognition, 
Identify the following type of contents in the document in JSON format:
```
TOOL: e.g. pot, spoon
INGREDIENT: e.g. mushroom, carrot
ACTIONS: e.g. fry
```

*Example:
add a liter of water to the pot, and boil the chicken to 100 degree
=>
```
'TOOL': 'pot',
'INGREDIENT': 'water', 'chicken',
'ACTIONS': 'boil'
```

Please transfer the following cookbook:
```
{input_form}
```
Return only the JSON format, no other words included
"""

prompt = PromptTemplate (
    template=template,
    input_variables=["input_form"]
)

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
user_input= "Add the chicken and toss to coat. Let stand at room temperature for 15 minutes, stirring occasionally."
reply = llm_chain.run(user_input)
print(reply)

## Knowlege graph memory

In [ ]:
!pip install networkx

In [ ]:
from langchain.chains.conversation.memory import ConversationKGMemory
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

llm = OpenAI (
    model_name='text-davinci-003',
    temperature=0,
    max_tokens = 256
)

template ="""
You are an expert of cooking and name entity recognition, 
Identify the following type of contents in the document in JSON format:
```
TOOL: e.g. pot, spoon
INGREDIENT: e.g. mushroom, carrot
ACTIONS: e.g. fry
```

*Example:
add a liter of water to the pot, and boil the chicken to 100 degree
=>
```
'TOOL': 'pot',
'INGREDIENT': 'water', 'chicken',
'ACTIONS': 'boil'
```

Please transfer the following cookbook:
```
{input}
```

Relevant Information:
```{history}```

Merge the TOOL,INGREDIENT,ACTIONS

Return only the JSON format, no other words included
"""

prompt = PromptTemplate (
    template=template,
    input_variables=["history","input"]
)
conversation_with_kg = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [ ]:
user_input= "Add the chicken and toss to coat. Let stand at room temperature for 15 minutes, stirring occasionally."

conversation_with_kg.predict(input=user_input)

In [ ]:
user_input= "Meanwhile, chop the bell pepper, celery, scallions, and garlic."
conversation_with_kg.predict(input=user_input)

In [ ]:
print(conversation_with_kg.memory.kg.get_triples ())

## EntityMemory

In [ ]:
from langchain import OpenAI, ConversationChain
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [ ]:
ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [ ]:
llm = OpenAI(
    model_name='text-davinci-003',
    temperature=0,
    max_tokens = 256
)

conversation_with_em = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [ ]:
conversation_with_em.predict(input="Hi I am going to cook GongBao chicken")

In [ ]:
conversation_with_em.memory.entity_store

## Reference
- Chat Markup Language:https://github.com/openai/openai-python/blob/main/chatml.md